In [3]:

config = {
    "azuregpt35-16k" : {
        "model_id":"gpt-35-turbo-16k", 
        "prompt_token_cost": 0.001,
        "response_token_cost": 0.002,
        "temperature": 1.0,
        "max_tokens": 2000,
        "stop": None,
        "api_version": "2023-07-01-preview",
        "api_base": "",
        "api_key": "",
        "embedding_id":"text-embedding-ada-002"
    },
    "memgraph" : {
        "host": "",
        "port": 7687
    },
    "weaviate" : {
        "api_key": "",
        "url": "",
        "db": "",
        "limit": 200
    }
}

from escargot import Escargot
question = "List the body parts/anatomy which over-express both the genes METTL5 and STYXL2"

# type(config) == dict
escargot = Escargot(config)

/Users/matsumoton/miniconda3/envs/escargot/lib/python3.9/site-packages/weaviate/__init__.py:130: DeprecationWarning: Dep010: Importing AuthApiKey from weaviate is deprecated. Import AuthApiKey from its module: weaviate.auth
  _Warnings.root_module_import(name, map_[name])


In [ ]:
from escargot import controller, language_models, operations

from escargot.vector_db.weaviate import WeaviateClient
import logging
import dill
import time
import sys 

question = "List the body parts/anatomy which over-express both the genes METTL5 and STYXL2"
# question = "Only answer if this is true or false: Basiliximab can treat Alzheimer's Disease"
# question = "Show me the drugs that can treat Alzheimer’s Disease"
question = "Only answer if this is true or false: Benzatropine can treat Alzheimer's Disease"
questions = ["Only answer if this is true or false: Benzatropine can treat Alzheimer's Disease",
"Only answer if this is true or false: Ropinirole can treat Alzheimer's Disease",
"Only answer if this is true or false: Haloperidol can treat Alzheimer's Disease",
"Only answer if this is true or false: Selegiline can treat Alzheimer's Disease",
"Only answer if this is true or false: Rivastigmine can treat Alzheimer's Disease",
"Only answer if this is true or false: Donepezil can treat Alzheimer's Disease",
"Only answer if this is true or false: Memantine can treat Alzheimer's Disease",
"Only answer if this is true or false: Quetiapine can treat Alzheimer's Disease",
"Only answer if this is true or false: Clozapine can treat Alzheimer's Disease",
"Only answer if this is true or false: Adenosine can treat Alzheimer's Disease",
"Only answer if this is true or false: Aripiprazole can treat Alzheimer's Disease",
"Only answer if this is true or false: Pioglitazone can treat Alzheimer's Disease",
"Only answer if this is true or false: Risperidone can treat Alzheimer's Disease",
"Only answer if this is true or false: Sunitinib can treat Alzheimer's Disease",
"Only answer if this is true or false: Tamoxifen can treat Alzheimer's Disease",
"Only answer if this is true or false: Vandetanib can treat Alzheimer's Disease",
"Only answer if this is true or false: Verapamil can treat Alzheimer's Disease",
"Only answer if this is true or false: Vorinostat can treat Alzheimer's Disease",
"Only answer if this is true or false: Vitamin E can treat Alzheimer's Disease",
"Only answer if this is true or false: Dexmethylphenidate can treat Alzheimer's Disease",
"Only answer if this is true or false: Ceforanide can treat Alzheimer's Disease",
"Only answer if this is true or false: Paliperidone can treat Alzheimer's Disease",
"Only answer if this is true or false: Triprolidine can treat Alzheimer's Disease",
"Only answer if this is true or false: Propranolol can treat Alzheimer's Disease",
"Only answer if this is true or false: Bepridil can treat Alzheimer's Disease",
"Only answer if this is true or false: Icosapent can treat Alzheimer's Disease",
"Only answer if this is true or false: Oxybutynin can treat Alzheimer's Disease",
"Only answer if this is true or false: Methylphenobarbital can treat Alzheimer's Disease",
"Only answer if this is true or false: Basiliximab can treat Alzheimer's Disease",
"Only answer if this is true or false: Muromonab can treat Alzheimer's Disease",
"Only answer if this is true or false: Trastuzumab can treat Alzheimer's Disease",
"Only answer if this is true or false: Rituximab can treat Alzheimer's Disease",
"Only answer if this is true or false: Ibritumomab tiuxetan can treat Alzheimer's Disease",
"Only answer if this is true or false: Digoxin Immune Fab (Ovine) can treat Alzheimer's Disease",
"Only answer if this is true or false: Hyaluronidase (ovine) can treat Alzheimer's Disease",
"Only answer if this is true or false: Insulin pork can treat Alzheimer's Disease",
"Only answer if this is true or false: Pancrelipase can treat Alzheimer's Disease",
"Only answer if this is true or false: Streptokinase can treat Alzheimer's Disease",
"Only answer if this is true or false: Clozapine can be considered for repurposing in the treatment of Alzheimer’s disease based on their potential efficacy",
"Only answer if this is true or false: Adenosine can be considered for repurposing in the treatment of Alzheimer’s disease based on their potential efficacy",
"Only answer if this is true or false: Aripiprazole can be considered for repurposing in the treatment of Alzheimer’s disease based on their potential efficacy",
"Only answer if this is true or false: Pioglitazone can be considered for repurposing in the treatment of Alzheimer’s disease based on their potential efficacy",
"Only answer if this is true or false: Risperidone can be considered for repurposing in the treatment of Alzheimer’s disease based on their potential efficacy",
"Only answer if this is true or false: Sunitinib can be considered for repurposing in the treatment of Alzheimer’s disease based on their potential efficacy",
"Only answer if this is true or false: Tamoxifen can be considered for repurposing in the treatment of Alzheimer’s disease based on their potential efficacy",
"Only answer if this is true or false: Vandetanib can be considered for repurposing in the treatment of Alzheimer’s disease based on their potential efficacy",
"Only answer if this is true or false: Verapamil can be considered for repurposing in the treatment of Alzheimer’s disease based on their potential efficacy",
"Only answer if this is true or false: Vorinostat can be considered for repurposing in the treatment of Alzheimer’s disease based on their potential efficacy",
"Only answer if this is true or false: Vitamin E can be considered for repurposing in the treatment of Alzheimer’s disease based on their potential efficacy",
"Only answer if this is true or false: Dexmethylphenidate can be considered for repurposing in the treatment of Alzheimer’s disease based on their potential efficacy",
"Only answer if this is true or false: Ceforanide can be considered for repurposing in the treatment of Alzheimer’s disease based on their potential efficacy",
"Only answer if this is true or false: Paliperidone can be considered for repurposing in the treatment of Alzheimer’s disease based on their potential efficacy",
"Only answer if this is true or false: Triprolidine can be considered for repurposing in the treatment of Alzheimer’s disease based on their potential efficacy",
"Only answer if this is true or false: Propranolol can be considered for repurposing in the treatment of Alzheimer’s disease based on their potential efficacy",
"Only answer if this is true or false: Bepridil can be considered for repurposing in the treatment of Alzheimer’s disease based on their potential efficacy",
"Only answer if this is true or false: Icosapent can be considered for repurposing in the treatment of Alzheimer’s disease based on their potential efficacy",
"Only answer if this is true or false: Oxybutynin can be considered for repurposing in the treatment of Alzheimer’s disease based on their potential efficacy",
"Only answer if this is true or false: Methylphenobarbital can be considered for repurposing in the treatment of Alzheimer’s disease based on their potential efficacy",
"Show me the drugs that can treat Alzheimer’s Disease",
"Show me the drugs that connect to Alzheimer’s Disease genes but do not connect to Alzheimer’s Disease",
"Show me the drugs that connect to body part that is  related to  Alzheimer’s Disease",
"Show me the drugs that satisfy the two points. 1) connect to Alzheimer’s Disease  genes but do not connect to Alzheimer’s Disease. 2) connect to body part that is  related to  Alzheimer’s Disease",
"Please provide a list of the 10 most promising drugs for repurposing in the treatment of Alzheimer’s disease based on their potential efficacy, and indicate the diseases they were originally developed to treat. Please rank them in descending order of potential effectiveness. Please check if the generated list meets the requirement: 1) exclude the drugs that were originally designed for Alzheimer’s disease, 2) 10 distinct drugs, 3) rank in descending order according to the potential effectiveness. If not, please regenerate the list that meets the requirement."
]

# question = "Please provide a list of the 10 most promising drugs for repurposing in the treatment of Alzheimer’s disease based on their potential efficacy, and indicate the diseases they were originally developed to treat. Please rank them in descending order of potential effectiveness. Please check if the generated list meets the requirement: 1) exclude the drugs that were originally designed for Alzheimer’s disease, 2) 10 distinct drugs, 3) rank in descending order according to the potential effectiveness. If not, please regenerate the list that meets the requirement."
# question = "Can Bepridil be considered for repurposing in the treatment of Alzheimer’s disease based on their potential efficacy and if so, why?"

answers = {}
for question in questions:

    def got() -> operations.GraphOfOperations:
        """
        Generates the Graph of Operations for the IO method.

        :return: Graph of Operations
        :rtype: GraphOfOperations
        """
        operations_graph = operations.GraphOfOperations()

        instruction_node = operations.Generate(1, 1)
        operations_graph.append_operation(instruction_node)
        
        return operations_graph
    # Retrieve the Graph of Operations
    got = got()

    config_file = "config.json"

    # Configure the Language Model (Assumes config.json is in the current directory with OpenAI API key)
    # lm = language_models.AzureGPT(config_file, model_name="azuregpt4o")
    lm = language_models.AzureGPT(config_file, model_name="azuregpt35-16k")
    vdb = WeaviateClient(config_file)

    # Create the Controller
    try:
        ctrl = controller.Controller(
        lm, 
        got, 
        ALZKBPrompter(vdb,lm), 
        ALZKBParser(),
        # The following dictionary is used to configure the initial thought state
        {
            "question": question,
            "input": "",
            "phase": "planning",
            "method" : "got"
        }
        )
        # ctrl.logger.addHandler(logging.StreamHandler(sys.stdout))
        # ctrl.logger.setLevel(logging.INFO)

        # Run the Controller and generate the output graph
        ctrl.run()
    except Exception as e:
        print("exception:",e)
    # delete the controller to free up memory

    # print(ctrl.graph.operations)
    #clear up logger
    ctrl.logger.handlers = []
    ctrl.logger = None

    operations_graph = ctrl.graph.operations
    del ctrl
    answers[question] = operations_graph
    print("question:",question)
    output = []
    operation = operations_graph[0]
    while len(operation.successors) > 0:
        operation = operation.successors[0]
    print(operation.get_thoughts()[0].state)
    break


In [ ]:
for question, operations_graph in answers.items():
    print("question:",question)
    output = []
    operation = operations_graph[0]
    while len(operation.successors) > 0:
        operation = operation.successors[0]
    print(operation.get_thoughts()[0].state['phase'], operation.get_thoughts()[0].state['input'])
    print("--------------------------------------------------")
    # break